In [6]:
!pip install requests beautifulsoup4 tqdm
!pip install selenium webdriver-manager



[notice] A new release of pip is available: 23.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 9.7/9.7 MB 29.6 MB/s eta 0:00:00
     ---------------------------------------- 126.3/126.3 kB ? eta 0:00:00
     ------------------------------------- 481.7/481.7 kB 29.5 MB/s eta 0:00:00
     -------------------------------------- 157.7/157.7 kB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 45.8/45.8 kB ? eta 0:00:00
     ---------------------------------------- 58.8/58.8 kB ? eta 0:00:00
     ---------------------------------------- 63.8/63.8 kB ? eta 0:00:00
INFO: pip is looking at multiple versions of requests to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 64.8/64.8 kB ? eta 0:00:00
     ---------------------------------------- 105.0/105.0 kB ? eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.0.1
    Uninstalling websocket-client-1.0.1:
      Successfully uninstalled webso

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.1.5 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.

[notice] A new release of pip is available: 23.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import json
import string
import time
import random
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException
from bs4 import BeautifulSoup

# Constantes
BASE_URL = "https://www.drugs.com"
ALPHA_URL_TEMPLATE = "https://www.drugs.com/alpha/{}.html"
OUTPUT_FILE = "drug_links.json"
OUTPUT_CSV = "drug_links.csv"
ERROR_LOG = "error_log.txt"

# Cargar datos previos si existen
all_links = []
urls_existentes = set()
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
        all_links = json.load(f)
        urls_existentes = set(link["url"] for link in all_links)
    print(f"📂 Cargados {len(all_links)} links previos desde {OUTPUT_FILE}")

# Setup del navegador con user-agent real
def create_driver():
    options = Options()
    options.headless = True
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) "
                         "Chrome/114.0.0.0 Safari/537.36")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Inicializar driver
driver = create_driver()

# Scraper de una combinación
def get_links_from_alpha(pair):
    global driver
    try:
        url = ALPHA_URL_TEMPLATE.format(pair)
        driver.get(url)
        time.sleep(random.uniform(5, 9))

        soup = BeautifulSoup(driver.page_source, "html.parser")
        links = []

        content = soup.find("div", id="content")
        ul_tags = content.find_all("ul") if content else []

        for ul in ul_tags:
            for li in ul.find_all("li"):
                a = li.find("a", href=True)
                if a and a["href"].startswith("/"):
                    full_url = BASE_URL + a["href"]
                    links.append({"drug_name": a.text.strip(), "url": full_url})
        return links
    except Exception as e:
        raise e

# Loop principal
errores = []
START_FROM = "ab"

full_combinaciones = [a + b for a in string.ascii_lowercase for b in string.ascii_lowercase]
start_index = full_combinaciones.index(START_FROM)
combinaciones = full_combinaciones[start_index:]

for i, pair in enumerate(combinaciones):
    print(f"🔠 Procesando: {pair}")
    try:
        links = get_links_from_alpha(pair)

        # Filtrar duplicados
        nuevos_links = [link for link in links if link["url"] not in urls_existentes]
        all_links.extend(nuevos_links)
        urls_existentes.update(link["url"] for link in nuevos_links)

        print(f"   ➕ {len(nuevos_links)} nuevos links encontrados.")
    except Exception as e:
        print(f"⚠️ Error en {pair}: {e}")
        errores.append(pair)

        try:
            driver.quit()
        except:
            pass
        time.sleep(random.uniform(3, 7))
        driver = create_driver()

    # Guardado progresivo cada 2 combinaciones
    if (i + 1) % 2 == 0:
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(all_links, f, ensure_ascii=False, indent=2)
        pd.DataFrame(all_links).to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
        print(f"💾 Guardado provisional luego de {i + 1} pares")
        time.sleep(random.uniform(1, 3))

# Guardado final
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(all_links, f, ensure_ascii=False, indent=2)
pd.DataFrame(all_links).to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

if errores:
    with open(ERROR_LOG, "w") as log:
        for e in errores:
            log.write(f"{e}\n")
    print(f"📝 Se registraron errores en {len(errores)} combinaciones. Ver {ERROR_LOG}")

driver.quit()
print(f"\n✅ Finalizado. Total drogas: {len(all_links)} guardadas en {OUTPUT_FILE} y {OUTPUT_CSV}")


🔠 Procesando: ab
   ➕ 72 nuevos links encontrados.
🔠 Procesando: ac
   ➕ 186 nuevos links encontrados.
💾 Guardado provisional luego de 2 pares
🔠 Procesando: ad
   ➕ 108 nuevos links encontrados.
🔠 Procesando: ae
   ➕ 5 nuevos links encontrados.
💾 Guardado provisional luego de 4 pares
🔠 Procesando: af
   ➕ 43 nuevos links encontrados.
🔠 Procesando: ag
   ➕ 10 nuevos links encontrados.
💾 Guardado provisional luego de 6 pares
🔠 Procesando: ah
   ➕ 4 nuevos links encontrados.
🔠 Procesando: ai
   ➕ 10 nuevos links encontrados.
💾 Guardado provisional luego de 8 pares
🔠 Procesando: aj
   ➕ 3 nuevos links encontrados.
🔠 Procesando: ak
   ➕ 26 nuevos links encontrados.
💾 Guardado provisional luego de 10 pares
🔠 Procesando: al
   ➕ 310 nuevos links encontrados.
🔠 Procesando: am
   ➕ 188 nuevos links encontrados.
💾 Guardado provisional luego de 12 pares
🔠 Procesando: an
   ➕ 151 nuevos links encontrados.
🔠 Procesando: ao
   ➕ 32 nuevos links encontrados.
💾 Guardado provisional luego de 14 pares
🔠